In [1]:
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup

import pandas as pd
import time
import re
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 동아일보 2024년 1월 1일부터 12월 31일 사이 '사다리'라는 키워드로 검색한 기사 수집하기
# 이 작업을 수행하기 전에 링크 전체 주소에 페이지 표시가 포함되어있는지 확인 필요

FIRST_PAGE = 'https://www.donga.com/news/search?p=1&query=%EC%82%AC%EB%8B%A4%EB%A6%AC&check_news=1&sorting=1&search_date=5&v1=20240101&v2=20241231&more=1'

# TARGET_URL 에서는 페이지 번호가 1부터 반복해서 입력될 수 있도록 제거(페이지 번호 위치가 어디인지 위, 아래 비교 필요)
TARGET_URL_KEYWORD = 'https://www.donga.com/news/search?p=' # 메인 + 페이지
TARGET_URL_UNTIL_DATE = '&query=%EC%82%AC%EB%8B%A4%EB%A6%AC&check_news=1&sorting=1&search_date=5&v1=20240101&v2=20241231&more=1' # 키워드 및 날짜
TARGET_URL = TARGET_URL_KEYWORD + TARGET_URL_UNTIL_DATE

print(TARGET_URL)

https://www.donga.com/news/search?p=&query=%EC%82%AC%EB%8B%A4%EB%A6%AC&check_news=1&sorting=1&search_date=5&v1=20240101&v2=20241231&more=1


## Collect article lists

In [ ]:
# 수집할 데이터를 저장할 빈 리스트를 생성
TITLE_OF_ARTICLE = [] # 기사 제목을 저장할 리스트
URL_OF_ARTICLE = []   # 기사 URL을 저장할 리스트
DATE_OF_ARTICLE = []  # 기사 게시 날짜를 저장할 리스트

# FIRST_PAGE URL에 대한 요청 객체를 생성하고, 사용자 에이전트를 설정
url = urllib.request.Request(FIRST_PAGE,
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
html = urlopen(url) # 생성한 요청 객체를 이용해 웹페이지를 열고 HTML을 가져옴
soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8') # BeautifulSoup를 사용해 HTML을 파싱

# 웹 페이지에서 기사 제목을 포함하는 div 태그의 클래스 'news_body' 내의 h4 태그 'tit'을 선택
for title in soup.select('div.news_body > h4.tit > a'): # 각 제목에 해당하는 a 태그 선택
    TITLE_OF_ARTICLE.append(title.get_text()) # a 태그의 텍스트(기사 제목)를 리스트에 추가

    article_URL = title['href'] # a 태그의 href 속성(기사 URL)을 리스트에 추가
    URL_OF_ARTICLE.append(article_URL)

# 웹 페이지에서 기사 날짜를 포함하는 span 태그 'date'를 선택
for date in soup.select('span.date'):
    date_of_article = date.get_text() # span 태그의 텍스트(기사 날짜)를 가져옴
    DATE_OF_ARTICLE.append(date_of_article)

# 추출한 기사 제목, URL, 날짜를 출력
print("\n".join(TITLE_OF_ARTICLE))
print("\n".join(URL_OF_ARTICLE))
print("\n".join(DATE_OF_ARTICLE))

일 잘하는 사람들이 보이지 않는 곳에서 반드시 하는 것 外 [책의향기 온라인]
GTX 파주~서울 22분·4450원…하루 5만명 실어 나른다
‘작업 중 경비원 사망’ 아파트 관리소장 금고 1년 선고
[광화문에서/박희창]불평등 키우는 ‘칩플레이션’… 성공 사다리 복원 지속해야
[횡설수설/박중현]엔진의 혼다-기술의 닛산 합병…새 도전 만난 현대차-기아
‘최동석 파경’ 박지윤 “언니·형부 초대로 크리스마스 홈파티”
11명의 ‘시민 영웅’, 보신각 제야의 종 울린다
흔들리는 ‘富의 사다리’, 청년 18% 소득계층 하락
위태로운 계층 이동 사다리…10명 중 7명 빈곤 탈출 못했다
[고양이 눈]2025년은 사다리 타듯
https://www.donga.com/news/Culture/article/all/20241227/130738925/1
https://www.donga.com/news/Economy/article/all/20241227/130737347/1
https://www.donga.com/news/Society/article/all/20241225/130718983/1
https://www.donga.com/news/Opinion/article/all/20241224/130717548/2
https://www.donga.com/news/Opinion/article/all/20241223/130710108/2
https://www.donga.com/news/Entertainment/article/all/20241223/130706617/1
https://www.donga.com/news/People/article/all/20241220/130686424/2
https://www.donga.com/news/Economy/article/all/20241219/130677160/2
https://www.donga.com/news/Economy/article/all/20241218/130670734/1
https://www.donga.com/news/Opinion/artic

In [ ]:
# 뉴스 제목으로부터 링크를 가져오는 함수 정의

def get_link_from_news_title(page_num, URL):
    for i in range(page_num):   # 페이지 수만큼 반복

        current_page_num = 10*i+1   # 현재 페이지 번호 계산 (동아일보 페이지 카운트 방식에 맞춰 계산)(동아일보는 페이지를 1, 11, 21 순으로 카운팅)
        position = URL.index('p=')      # URL에서 'p=' 위치 찾기
        URL_with_page_num = URL[:position+2] + str(current_page_num) + URL[position+2:]     # 현재 페이지 번호를 URL에 추가
        # 요청 객체 생성 및 헤더 설정
        url = urllib.request.Request(URL_with_page_num,
                                        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})

        html = urlopen(url)     # 요청 객체를 이용해 웹페이지 열기 및 HTML 가져오기
        soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')    # BeautifulSoup을 사용해 HTML 파싱

        # 웹 페이지에서 기사 제목을 포함하는 div 태그의 클래스 'news_body' 내의 h4 태그 'tit'을 선택
        for title in soup.select('div.news_body > h4.tit > a'): # 각 제목에 해당하는 a 태그 선택
            TITLE_OF_ARTICLE.append(title.get_text()) # a 태그의 텍스트(기사 제목)를 리스트에 추가

            article_URL = title['href'] # a 태그의 href 속성(기사 URL)을 리스트에 추가
            URL_OF_ARTICLE.append(article_URL)

        # 웹 페이지에서 기사 날짜를 포함하는 span 태그 'date'를 선택
        for date in soup.select('span.date'):
            date_of_article = date.get_text() # span 태그의 텍스트(기사 날짜)를 가져옴
            DATE_OF_ARTICLE.append(date_of_article)

        if (i) % 10 == 0:   # 매 10번째 루프마다
            time.sleep(2)   # 서버 과부하 방지를 위한 2초 대기
            print(f'{(i) * 10} articles complete')     # 진행 상황 출력

In [ ]:
# 전체 기사 수를 받아 필요한 페이지 수를 계산하는 함수 정의
# 아래 코드는 내가 보고있는 페이지의 게시물 개수에 따라 숫자가 달라지므로 반드시 확인

def get_total_num_of_article(number):
    if(number % 10 >= 1):   # 10으로 나눈 나머지가 1 이상이면 (즉, 나누어떨어지지 않으면)
        page_num = int((number / 10) + 1)   # 몫에 1을 더하여 추가 페이지 확보 (반올림 효과)
    else:   # 10으로 나누어떨어지는 경우
        page_num = int(number / 10)     # 정확히 나눈 몫이 곧 페이지 수
    return page_num

page_num = get_total_num_of_article(249)
page_num

25

In [ ]:
# 기사 제목, URL, 게시 날짜를 저장할 리스트를 초기화
TITLE_OF_ARTICLE = []
URL_OF_ARTICLE = []
DATE_OF_ARTICLE = []

# get_link_from_news_title 함수를 호출하여 지정된 페이지 수와 URL을 기반으로 뉴스 기사의 제목, URL, 게시 날짜를 수집
get_link_from_news_title(page_num, TARGET_URL)

0th article complete
100th article complete
200th article complete


In [ ]:
df = pd.DataFrame(list(zip(DATE_OF_ARTICLE, URL_OF_ARTICLE, TITLE_OF_ARTICLE)), columns =['Date', 'Url', 'Title'])  # 수집한 리스트를 각각 (날짜, URL, 제목) 튜플로 묶은 후 데이터프레임 처리

print(len(df))  # 생성된 DataFrame의 행 개수 출력 (=기사 수)
df  # 전체 DataFrame 출력

249


,Date,Url,Title
0,2024-12-27,https://www.donga.com/news/Culture/article/all...,일 잘하는 사람들이 보이지 않는 곳에서 반드시 하는 것 外 [책의향기 온라인]
1,2024-12-27,https://www.donga.com/news/Economy/article/all...,GTX 파주~서울 22분·4450원…하루 5만명 실어 나른다
2,2024-12-25,https://www.donga.com/news/Society/article/all...,‘작업 중 경비원 사망’ 아파트 관리소장 금고 1년 선고
3,2024-12-24,https://www.donga.com/news/Opinion/article/all...,[광화문에서/박희창]불평등 키우는 ‘칩플레이션’… 성공 사다리 복원 지속해야
4,2024-12-23,https://www.donga.com/news/Opinion/article/all...,[횡설수설/박중현]엔진의 혼다-기술의 닛산 합병…새 도전 만난 현대차-기아
...,...,...,...
244,2024-01-04,https://www.donga.com/news/Society/article/all...,추락사 현장에 ‘피 묻은 안전모’ 슬쩍…중대재해 조작·은폐 발각
245,2024-01-03,https://www.donga.com/news/Opinion/article/all...,[HBR INSIGHT]직원들이 기업의 핵심 가치를 기억하게 하려면
246,2024-01-03,https://www.donga.com/news/Economy/article/all...,“금투세 폐지”에 개미들 환영… “공매도 금지 이은 총선용” 지적도
247,2024-01-02,https://www.donga.com/news/Politics/article/al...,尹 “금융투자소득세 폐지 추진”…공매도 개혁도


In [ ]:
# DataFrame을 CSV 파일로 저장
df.to_csv("/content/drive/MyDrive/CLASS/202502_NLP/Donga.csv",     # 저장할 파일 경로 (Google Drive 내 지정 경로)
    index=False,    # 행 번호(인덱스)를 저장하지 않음
    encoding='utf-8-sig'    # 인코딩 방식: 한글 깨짐 방지를 위해 'utf-8-sig' 사용
)

## Collect texts from the article

In [ ]:
# 기사 본문 수집 시도해보기

url = urllib.request.Request(URL_OF_ARTICLE[2],     # 특정 기사의 URL을 지정
                             # 사용자 에이전트 설정으로 웹 서버가 이 요청을 웹 브라우저에서 온 것으로 인식하게 함
                             headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})

html = urllib.request.urlopen(url)  # URL을 열어 HTML을 가져옴

soup = BeautifulSoup(html, 'html.parser', from_encoding='utf-8')    # BeautifulSoup을 사용하여 HTML 문서를 파싱
content = soup.select('section.news_view')      # 기사 내용이 담긴 div 태그의 클래스 'article_txt'를 선택

CONTENT = content[0].get_text()     # 선택된 요소의 텍스트 내용을 추출
CONTENT = CONTENT.replace('\n', ' ').replace('\r', '')      # 동아일보는 텍스트 인코딩 문제 때문에 \r 을 추가로 제거

# 최종적으로 정제된 기사 내용을 출력
CONTENT

' 재판부 “피해자 사망하는 중대한 결과 발생” 법정구속은 면해…피고인 A씨, 항소장 제출     ⓒ뉴시스 아파트 경비원이 조경수 전지작업을 하다 추락해 사망한 사건 관련 안전조치 없이 작업을 하도록 방치한 혐의로 재판에 넘겨진 관리소장에 대해 금고형이 선고됐다.25일 법조계에 따르면 의정부지법 형사2단독 김상현 판사는 업무상과실치사 혐의로 불구속 기소된 아파트 관리소장 A씨에 대해 금고 1년을 선고했다.금고형은 징역처럼 교도소에 수감되는 것은 동일하지만 징역과 달리 노역이 강제되지 않는 형벌이다.  경기 의정부시 한 아파트 관리사무소장인 A씨는 지난 3월11일 같은 아파트 소속 70대 경비원 B씨가 추락방지 안전장치 없이 사다리를 타고 조경수 전지작업을 하도록 방치해 숨지게 한 혐의다.B씨는 작업과정에서 다리를 헛디뎌 약 2.5m 아래로 추락해 다발성 장기 부전으로 숨졌다.사고 당시 B씨는 안전모를 착용하고 있었지만 추락방지를 위한 안전고리 등 보호장구는 착용하고 있지 않았던 것으로 조사됐다.경비원에 대한 관리·감독자인 관리소장은 사고를 방지해야 하는 업무상 주의의무가 있다.  재판부는 “피해자의 사망이라는 중대한 결과가 발생한 점, 피해자 측과 합의하지 못한 점, 범행을 인정하고 반성하는 점 등 여러 양형조건을 종합해 형을 정했다”고 밝혔다.또 피해자 유족들과 추가적인 합의 기회를 부여하기 위해 법정구속은 하지 않았다.A씨는 판결에 불복해 법원에 항소장을 제출한 상태다.[의정부=뉴시스]                     '

In [ ]:
# 기사 URL 목록을 입력으로 받아 각 URL에서 텍스트를 추출하는 함수

def get_texts_from_news(URL):
    # URL 목록의 각 URL에 대하여 반복 실행
    for i in range(len(URL)):
        url = urllib.request.Request(URL[i], headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.85 Safari/537.36'})
        source_code_from_url = urllib.request.urlopen(url)
        soup = BeautifulSoup(source_code_from_url, 'html.parser', from_encoding='utf-8')
        # content가 에 들어갈 내용이 존재하면
        try:
            content = soup.select('section.news_view') # 기사 본문이 담긴 div 태그의 클래스 'article_txt'에서 텍스트 내용 추출
            CONTENT = content[0].get_text().replace('\n', ' ').replace('\r', '') # 선택된 요소의 텍스트 내용을 추출하고, 개행 문자 제거
            CONTENT_OF_ARTICLE.append(CONTENT) # 정제된 기사 본문을 리스트에 추가

        # content가 존재하지 않으면
        except Exception as e: # 코드가 진행되지 않는 상황의 경우,
            print(f"Failed to process the {i+1}th article: {URL}") # 에러가 발생한 기사가 몇 번째인지와 링크 출력
            CONTENT = 'NONE' # 기사가 삭제되거나 확인할 수 없는 경우 기사 본문 대신 'NONE'이라는 텍스트 대신 입력
            CONTENT_OF_ARTICLE.append(CONTENT)

        # 서버 부하 방지 및 진행 모니터링
        if i % 10 == 0:
            time.sleep(1)    # 10개마다 1초 대기
        if i % 100 == 0:    # 100개마다 진행 상태 출력
            print(f'{i}nd article complete')

In [ ]:
CONTENT_OF_ARTICLE = []

get_texts_from_news(URL_OF_ARTICLE)

0nd article complete
100nd article complete
200nd article complete


In [ ]:
df = pd.DataFrame(list(zip(DATE_OF_ARTICLE, URL_OF_ARTICLE, TITLE_OF_ARTICLE, CONTENT_OF_ARTICLE)), columns =['Date', 'Url', 'Title', 'Content'])

print(len(df))
df.head()

249


,Date,Url,Title,Content
0,2024-12-27,https://www.donga.com/news/Culture/article/all...,일 잘하는 사람들이 보이지 않는 곳에서 반드시 하는 것 外 [책의향기 온라인],...
1,2024-12-27,https://www.donga.com/news/Economy/article/all...,GTX 파주~서울 22분·4450원…하루 5만명 실어 나른다,배차간격 10분…기본요금 3200원에 거리 요금 추가 첫차 오전 5시 30분·막차...
2,2024-12-25,https://www.donga.com/news/Society/article/all...,‘작업 중 경비원 사망’ 아파트 관리소장 금고 1년 선고,"재판부 “피해자 사망하는 중대한 결과 발생” 법정구속은 면해…피고인 A씨, 항소장..."
3,2024-12-24,https://www.donga.com/news/Opinion/article/all...,[광화문에서/박희창]불평등 키우는 ‘칩플레이션’… 성공 사다리 복원 지속해야,박희창 경제부 차장 ‘○○플레이션’이 유행하더니 이제는 ‘칩플레이션(Che...
4,2024-12-23,https://www.donga.com/news/Opinion/article/all...,[횡설수설/박중현]엔진의 혼다-기술의 닛산 합병…새 도전 만난 현대차-기아,1973년 아랍석유수출국기구(OAPEC) 회원국의 원유 금수 조치로 시작된...


In [ ]:
df['Content'].replace('NONE', np.nan, inplace=True) # 'Content' 열에서 'NONE' 문자열을 NaN 값으로 대체
df['Content'].replace('', np.nan, inplace=True) # 'Content' 열에서 빈 문자열을 NaN 값으로 대체
df.dropna(subset=['Content'], inplace=True) # 'Content' 열에 NaN 값을 포함하는 모든 행을 삭제

# 처리 후 남은 행의 개수 출력
print(len(df))

249


/tmp/ipython-input-1632476055.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Content'].replace('NONE', np.nan, inplace=True) # 'Content' 열에서 'NONE' 문자열을 NaN 값으로 대체
/tmp/ipython-input-1632476055.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

In [ ]:
df = df[~df["Content"].str.contains(r"사다리차",     # "사다리차"라는 단어가 "Content" 컬럼에 포함된 행을 필터링
                                    case=False,     # 대소문자 구분 없이 검색
                                    na=False)]      # 결측값(NaN)은 False로 간주하여 포함되지 않도록 설정

print(len(df))
df.to_csv("/content/drive/MyDrive/CLASS/202502_NLP/Donga_final.csv",index=False, encoding='utf-8-sig')

237
